# 영업 성공 여부 분류 경진대회

## 1. 데이터 확인

### 필수 라이브러리

In [87]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score


### 데이터 셋 읽어오기

In [88]:
df_train = pd.read_csv("data/encoded_train.csv") # 학습용 데이터
df_test = pd.read_csv("data/encoded_submission.csv") # 테스트 데이터(제출파일의 데이터)

In [89]:
df_train.head() # 학습용 데이터 살펴보기

,bant_submit,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,historical_existing_cnt,id_strategic_ver,it_strategic_ver,idit_strategic_ver,customer_job,...,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted,enterprise_Enterprise,enterprise_SMB
0,1.0,0.060187,0.066667,32160,0.116088,0,0,0,0,0.143617,...,1,0,0.003079,0.026846,0.064119,0.123188,0,1,True,False
1,1.0,0.060187,0.066667,23122,0.116088,12,0,0,0,0.076423,...,1,0,0.003079,0.026846,0.064119,0.084211,1,1,True,False
2,1.0,0.060283,0.088889,1755,0.112788,144,0,0,0,0.093429,...,1,0,0.003079,0.026846,0.067235,0.118421,2,1,True,False
3,1.0,0.059962,0.088889,4919,0.109474,0,0,0,0,0.088910,...,1,0,0.003079,0.026846,0.064915,0.132075,3,1,True,False
4,1.0,0.060107,0.088889,17126,0.078316,0,0,0,0,0.098375,...,0,0,0.003079,0.026846,0.064232,0.082825,4,1,True,False


## 2. 데이터 전처리

### 레이블 인코딩

다시 학습 데이터와 제출 데이터를 분리합니다.

### 2-2. 학습, 검증 데이터 분리

In [90]:
x_train, x_val, y_train, y_val = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=400
)

In [91]:
df_test.drop(columns='id', axis=1, inplace=True)
x_submission = df_test.drop("is_converted", axis=1)
y_submission = df_test["is_converted"]

## 3. 모델 학습

### 모델 정의 

In [92]:
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

In [93]:
model.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

### 모델 학습

### 모델 성능 보기

In [94]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [95]:
pred = model.predict(x_submission.fillna(0))
get_clf_eval(y_submission, pred)

오차행렬:
 [[ 959  222]
 [1623 2467]]

정확도: 0.6500
정밀도: 0.3714
재현율: 0.8120
F1: 0.5097


## 4. 제출하기

### 테스트 데이터 예측

In [96]:
test_pred = model.predict(x_submission.fillna(0))
sum(test_pred) # True로 예측된 개수

2582

### 제출 파일 작성

In [97]:
# # 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
# df_sub = pd.read_csv("./data/encoded_submission.csv")
# df_sub["is_converted"] = test_pred

# # 제출 파일 저장
# df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**